In [ ]:
# body reconstruction
#
# finds rectangular shapes
# understand which ones could be a box
# indentifies panel's body

In [ ]:
# OpenCV + Python Rectangle Detection Example
# https://www.youtube.com/watch?v=T9P10FAp8Qw
# 1) https://mehmethanoglu.com.tr/blog/6-opencv-ile-dikdortgen-algilama-python.html
# https://youtu.be/HpaY0HOomAI
# # https://github.com/shibatch/rectdetect
# https://youtu.be/Fchzk1lDt7Q
# https://docs.opencv.org/master/d3/dc0/group__imgproc__shape.html
# https://stackoverflow.com/questions/44522012/rectangle-detection-tracking-using-opencv
# https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html
# https://docs.opencv.org/3.4/da/d0c/tutorial_bounding_rects_circles.html
# https://cv-tricks.com/opencv-dnn/edge-detection-hed/
# https://arxiv.org/pdf/1703.10896.pdf
# https://www.youtube.com/watch?v=zzcipxzZP9E

**To make an optical form reader it is necessary to follow the necessary steps sequentially.**
1. First, we have to successfully pass the rectangle detection function. The operations performed for this are logically simple, but small adjustments lead to a lot of changes;
2. we transform a colored picture into gray, that is, gray and its tones;
3. Then we apply a bilateral filter. Every process we do is to make the objects more perceptible through the gray colored picture. For this purpose, removing small details etc. etc.
4. After eliminating the small details a little, we will use the Canny function to get corners or waypoints. With this filter, we will get a completely black and white result. (Detailed explanation about Canny function)
5. In the next steps, the geometry we are looking for is a rectangle and similar shapes will be obtained by contour and looking at the number of corners. If it has 4 corners, it will be a suitable geometry for us. We will convert the rectangle into perspective -> normal view according to the coordinates we have obtained. Thus, we will have the quadrilateral in a normal view.

In [6]:
import cv2
import time
import numpy as np

##################
DELAY = 0.02
USE_CAM = 1
IS_FOUND = 0

MORPH = 7
CANNY = 250
##################
# 420x600 oranı 105mmx150mm gerçek boyuttaki kağıt için
_width  = 600.0
_height = 420.0
_margin = 0.0
##################

if USE_CAM: video_capture = cv2.VideoCapture(0)

corners = np.array(
    [
    [[  _margin, _margin]],
    [[ _margin, _height + _margin  ]],
    [[ _width + _margin, _height + _margin  ]],
    [[ _width + _margin, _margin ]],
    ]
)

pts_dst = np.array( corners, np.float32 )

while True :

    if USE_CAM :
        ret, rgb = video_capture.read()
    else :
        ret = 1
        rgb = cv2.imread( "opencv.jpg", 1 )

    if ( ret ):

        gray = cv2.cvtColor( rgb, cv2.COLOR_BGR2GRAY )

        gray = cv2.bilateralFilter( gray, 1, 10, 120 )

        edges  = cv2.Canny( gray, 10, CANNY )

        kernel = cv2.getStructuringElement( cv2.MORPH_RECT, ( MORPH, MORPH ) )

        closed = cv2.morphologyEx( edges, cv2.MORPH_CLOSE, kernel )
        contours, h = cv2.findContours( closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE )

        for cont in contours:

            # Küçük alanları pass geç
            if cv2.contourArea( cont ) > 5000 :

                arc_len = cv2.arcLength( cont, True )

                approx = cv2.approxPolyDP( cont, 0.1 * arc_len, True )

                if ( len( approx ) == 4 ):
                    IS_FOUND = 1
                    #M = cv2.moments( cont )
                    #cX = int(M["m10"] / M["m00"])abs
                    #cY = int(M["m01"] / M["m00"])
                    #cv2.putText(rgb, "Center", (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

                    pts_src = np.array( approx, np.float32 )

                    h, status = cv2.findHomography( pts_src, pts_dst )
                    out = cv2.warpPerspective( rgb, h, ( int( _width + _margin * 2 ), int( _height + _margin * 2 ) ) )

                    cv2.drawContours( rgb, [approx], -1, ( 255, 0, 0 ), 2 )

                else : pass

        #cv2.imshow( 'closed', closed )
        #cv2.imshow( 'gray', gray )
        cv2.namedWindow( 'edges', cv2.CV_WINDOW_AUTOSIZE )
        cv2.imshow( 'edges', edges )

        cv2.namedWindow( 'rgb', cv2.CV_WINDOW_AUTOSIZE )
        cv2.imshow( 'rgb', rgb )

        if IS_FOUND :
            cv2.namedWindow( 'out', cv2.CV_WINDOW_AUTOSIZE )
            cv2.imshow( 'out', out )

        if cv2.waitKey(27) & 0xFF == ord('q') :
            break

        if cv2.waitKey(99) & 0xFF == ord('c') :
            current = str( time.time() )
            cv2.imwrite( 'ocvi_' + current + '_edges.jpg', edges )
            cv2.imwrite( 'ocvi_' + current + '_gray.jpg', gray )
            cv2.imwrite( 'ocvi_' + current + '_org.jpg', rgb )
            print("Pictures saved")

        time.sleep( DELAY )

    else :
        print "Stopped"
        break

if USE_CAM : video_capture.release()
cv2.destroyAllWindows()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Pictures saved")? (<ipython-input-6-6d9f7101bda1>, line 113)